In [22]:
%%capture
!pip install fcd

In [23]:
!mkdir generated_smiles -p
!wget https://raw.githubusercontent.com/bioinf-jku/FCD/master/generated_smiles/LSTM_Segler.smi -O generated_smiles/LSTM_Segler.smi

--2024-04-01 18:53:15--  https://raw.githubusercontent.com/bioinf-jku/FCD/master/generated_smiles/LSTM_Segler.smi
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.109.133, 185.199.108.133, 185.199.111.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.109.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 22730454 (22M) [text/plain]
Saving to: ‘generated_smiles/LSTM_Segler.smi’

generated_smiles/LS 100%[===================>]  21.68M  3.07MB/s    in 7.3s    

2024-04-01 18:53:22 (2.98 MB/s) - ‘generated_smiles/LSTM_Segler.smi’ saved [22730454/22730454]



In [24]:
import os
from rdkit import RDLogger
import numpy as np

from fcd import get_fcd, load_ref_model, canonical_smiles, get_predictions, calculate_frechet_distance

RDLogger.DisableLog("rdApp.*")

np.random.seed(0)
os.environ["CUDA_VISIBLE_DEVICES"] = "0"  # set gpu

### Load and prepare data
Take care when preparing data.
- Different canonicalizations change the FCD. rdkit canonicalization should be used.
- In case the generated "molecules" contain invalid SMILES, decide if you want to include them in the FCD.
- Make sure that you respect sample sizes as the FCD varies with size.

In [25]:
# Load chemnet model
model = load_ref_model()

# Load generated molecules
gen_mol_file = "generated_smiles/LSTM_Segler.smi"  # input file which contains one generated SMILES per line
with open(gen_mol_file) as f:
    gen_mol = f.read().split("\n")

# IMPORTANT: take at least 10000 molecules as FCD can vary with sample size
sample1 = np.random.choice(gen_mol, 10000, replace=False)
sample2 = np.random.choice(gen_mol, 10000, replace=False)

# get canonical smiles and filter invalid ones
can_sample1 = [w for w in canonical_smiles(sample1) if w is not None]
can_sample2 = [w for w in canonical_smiles(sample2) if w is not None]

## Calculation of FCD

In [26]:
# get CHEBMLNET activations of generated molecules
act1 = get_predictions(model, can_sample1)
act2 = get_predictions(model, can_sample2)

mu1 = np.mean(act1, axis=0)
sigma1 = np.cov(act1.T)

mu2 = np.mean(act2, axis=0)
sigma2 = np.cov(act2.T)

fcd_score = calculate_frechet_distance(mu1=mu1, mu2=mu2, sigma1=sigma1, sigma2=sigma2)

print("FCD: ", fcd_score)

FCD:  0.3298386855756661


In [27]:
"""if you don't need to store the activations you can also take a shortcut."""
fcd_score = get_fcd(can_sample1, can_sample2, model)

print("FCD: ", fcd_score)

FCD:  0.3298386855756661


In [28]:
"""This is what happens if you do not canonicalize the smiles"""
fcd_score = get_fcd(can_sample1, sample2, model)
print("FCD: ", fcd_score)

FCD:  25.552174526889033
